In [15]:
import lightgbm as lgb

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics
from sklearn import decomposition
import math

import re
%matplotlib inline

In [5]:
def read_data(filename):
    data = pd.read_csv(filename, header=0, sep=',', encoding='pt154')
    data = data.rename(index=str, columns={'client_id': '_ID_', 'open_acount_flg': '_VAL_'})
    return data

In [6]:
train = read_data('../data/dummy_encoding_all_city_train.csv')
test = read_data('../data/dummy_encoding_all_city_test.csv')

In [7]:
train.head()

,_ID_,age,credit_sum,score_shk,monthly_income,credit_count,overdue_credit_count,open_account_flg,gender=0,gender=1,...,living_region=78,living_region=79,living_region=80,living_region=81,living_region=82,living_region=83,living_region=84,living_region=85,living_region=86,living_region=87
0,1,48,60.0,0.770249,30000.0,1.0,1.0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,28,11.0,0.248514,43000.0,2.0,0.0,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,32,11.0,0.459589,23000.0,5.0,0.0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,27,12.0,0.362536,17000.0,2.0,0.0,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,45,17.0,0.421385,25000.0,1.0,0.0,0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
y = train.open_account_flg
del train['_ID_']
del train['open_account_flg']

In [9]:
client_ids = test._ID_
del test['_ID_']

In [10]:
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(train, y, test_size=0.3)

In [11]:
X_test = test

In [12]:
print(X_train.shape, X_valid.shape, X_test.shape)

(119522, 187) (51224, 187) (91940, 187)


# Lightgbm

### parameter information: https://github.com/dmlc/xgboost/blob/master/doc/parameter.md

In [25]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_valid, y_valid)

In [54]:
params = {
    'task': 'train',
    'num_trees': 400,
#     'num_leaves': 250,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'auc',
    'max_depth': 7,
    'learning_rate': 0.1,
    'sub_feature': 0.8,
    'sub_row': 0.8,
#     'lambda_l1': 2,
#     'lambda_l2': 2,
    'verbose': 0
}

In [55]:
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_valid,
                early_stopping_rounds=25)

[1]	valid_0's auc: 0.731938
Train until valid scores didn't improve in 25 rounds.
[2]	valid_0's auc: 0.73364
[3]	valid_0's auc: 0.738123
[4]	valid_0's auc: 0.739823
[5]	valid_0's auc: 0.741026
[6]	valid_0's auc: 0.741837
[7]	valid_0's auc: 0.742728
[8]	valid_0's auc: 0.742944
[9]	valid_0's auc: 0.744242
[10]	valid_0's auc: 0.744514
[11]	valid_0's auc: 0.745378
[12]	valid_0's auc: 0.745914
[13]	valid_0's auc: 0.746567
[14]	valid_0's auc: 0.746892
[15]	valid_0's auc: 0.747315
[16]	valid_0's auc: 0.747554
[17]	valid_0's auc: 0.747601
[18]	valid_0's auc: 0.748155
[19]	valid_0's auc: 0.748539
[20]	valid_0's auc: 0.748755
[21]	valid_0's auc: 0.749234
[22]	valid_0's auc: 0.749499
[23]	valid_0's auc: 0.749755
[24]	valid_0's auc: 0.750095
[25]	valid_0's auc: 0.750276
[26]	valid_0's auc: 0.750606
[27]	valid_0's auc: 0.750961
[28]	valid_0's auc: 0.751174
[29]	valid_0's auc: 0.751445
[30]	valid_0's auc: 0.751556
[31]	valid_0's auc: 0.75165
[32]	valid_0's auc: 0.75205
[33]	valid_0's auc: 0.752174
[

In [99]:
pred_test = xgb1.predict_proba(X_test)[:, 1]
prediction = pd.DataFrame(data={'_ID_': client_ids, '_VAL_': pred_test}, index=test.index)
prediction.to_csv('../data/prediction.csv', index=False)
print(prediction.head())

     _ID_     _VAL_
0  170747  0.058535
1  170748  0.137427
2  170749  0.280323
3  170750  0.158912
4  170751  0.067224
